# Reinforcement Learning 


## **Problem 1 - Pick-and-Place Robot as an MDP**

> **What is the goal?**

We want a robot arm to pick up an object and place it somewhere. It should do this in a way that is fast and smooth.

To make this happen, we use Reinforcement Learning (RL) — and first, we turn this into a Markov Decision Process (MDP).

> **MDP Components (What RL needs to learn):**

States (S) – What the robot knows about the world right now.

Actions (A) – What the robot can do next.

Rewards (R) – How good or bad the action was.

Transitions (T) – What happens when you do an action in a state.

Goal – Learn a policy (way to act) to maximize rewards over time.

### Step 1: Define States (S)

The state is the situation the robot is in.

State can include:

Position of robot arm joints (angles)

Velocity of each joint

Position of the object to be picked

Position of where to place it

### Example:

S = [joint1_angle, joint1_velocity, joint2_angle, joint2_velocity, object_position, target_position]

---



### Step 2: Define Actions (A)

The actions are what the robot can do.

Actions could be:

Increase or decrease motor speed

Move joint slightly left or right

Open or close the gripper

**Example:**

A = [move_joint1_left, move_joint1_right, move_joint2_left, move_joint2_right, open_gripper, close_gripper]

---

### Step 3: Define Rewards (R)

Rewards tell the robot if it did good or bad.

+10 → If the object is successfully placed at the target

+5 → If the object is picked up

-1 → If the movement is jerky or slow

-5 → If the robot drops the object or misses

---

### Step 4: Transitions (T)

What happens when an action is taken in a state.

In this robot, transitions are deterministic (we can predict what happens)

For example:

If joint1 moves right by 5°, then next angle = angle + 5°

### Final MDP Summary:

States (S):
    Positions and velocities of robot arm joints, object location, target location

Actions (A):
    Small motor commands: move joints, open/close gripper

Rewards (R):
    +10 for placing object correctly
    +5 for picking object
    -1 for slow/jerky moves
    -5 for dropping/missing object

Transitions:
    Deterministic - robot knows exactly what happens after each action

Goal:
    Learn to pick and place smoothly and quickly using RL.

### Problem 2 [20 Marks] - 2x2 Gridworld & Value Iteration

>The Grid:

We have a 2x2 grid with 4 states:

s1  s2
s3  s4


Actions = up, down, left, right
Rewards:

s1 = +5

s2 = +10

s3 = +1

s4 = +2

Initial policy: always move UP.

🔁 Value Iteration - What is it?

Value Iteration helps us figure out the best value of each state.
We update the value of each state based on:

V(s) = max over a [ sum of (reward + discounted future value) ]


Let’s use:

Discount factor γ = 1 (for simplicity)

----

### **Iteration 1:**

Step 1: Initial Values

Let’s say initially:

V(s1) = 0
V(s2) = 0
V(s3) = 0
V(s4) = 0

Step 2: Update Each State's Value

We calculate value of each state by looking at the reward from going to the next state.

Let’s assume:

Valid moves take you to next state

Invalid moves = stay in the same state

Let’s update:

s1:

Up → stays in s1 → reward = 5 + V(s1) = 5 + 0 = 5

s2:

Up → stays in s2 → reward = 10 + V(s2) = 10 + 0 = 10

s3:

Up → goes to s1 → reward = 5 + V(s1) = 5 + 0 = 5

s4:

Up → goes to s2 → reward = 10 + V(s2) = 10 + 0 = 10

✅ Updated Values after Iteration 1:
V(s1) = 5
V(s2) = 10
V(s3) = 5
V(s4) = 10

🔁 Iteration 2:

Now use updated values:

s1:

Up → s1 → 5 + V(s1) = 5 + 5 = 10

s2:

Up → s2 → 10 + V(s2) = 10 + 10 = 20

s3:

Up → s1 → 5 + V(s1) = 5 + 5 = 10

s4:

Up → s2 → 10 + V(s2) = 10 + 10 = 20

✅ Updated Values after Iteration 2:
V(s1) = 10
V(s2) = 20
V(s3) = 10
V(s4) = 20

### Problem 3 – 5x5 Gridworld using Value Iteration

🗺️ Gridworld Setup:

We have a 5x5 grid, like this (rows and columns like a matrix):

s0,0  s0,1  s0,2  s0,3  s0,4
s1,0  s1,1  s1,2  s1,3  s1,4
s2,0  s2,1  s2,2  s2,3  s2,4
s3,0  s3,1  s3,2  s3,3  s3,4
s4,0  s4,1  s4,2  s4,3  s4,4

🧠 What is special about it?

Goal state = s4,4 → Reward = +10

Grey bad states = s0,4, s2,2, s3,0 → Reward = −5

All other states → Reward = −1

🎮 Actions:

Each state allows these 4 moves:

→ (right)

↓ (down)

← (left)

↑ (up)

If move is invalid (hits wall), the agent stays in the same state.

🏗️ Task 1: Update MDP code

We’ll write code that:

Builds the 5x5 environment

Assigns rewards to each state

Runs Value Iteration

Shows:

V* (best values)

π* (best policy → direction to move)

In [7]:

import numpy as np

# Grid size
rows, cols = 5, 5

# All states
states = [(i, j) for i in range(rows) for j in range(cols)]

# Goal and bad (grey) states
goal_state = (4, 4)
grey_states = [(0, 4), (2, 2), (3, 0)]

# Rewards
def get_reward(state):
    if state == goal_state:
        return 10
    elif state in grey_states:
        return -5
    else:
        return -1

# All possible actions
actions = {
    'up': (-1, 0),
    'down': (1, 0),
    'left': (0, -1),
    'right': (0, 1)
}

# Discount factor
gamma = 0.9

# Initialize value function
V = np.zeros((rows, cols))

# Value Iteration function
def value_iteration(V, threshold=1e-4):
    policy = np.full((rows, cols), '', dtype=object)
    iteration = 0

    while True:
        delta = 0
        new_V = V.copy()

        for state in states:
            if state == goal_state:
                continue  # skip terminal state

            i, j = state
            max_value = float('-inf')
            best_action = None

            for action, move in actions.items():
                ni, nj = i + move[0], j + move[1]

                # Stay in the same state if hitting wall
                if 0 <= ni < rows and 0 <= nj < cols:
                    next_state = (ni, nj)
                else:
                    next_state = (i, j)

                reward = get_reward(next_state)
                value = reward + gamma * V[next_state]

                if value > max_value:
                    max_value = value
                    best_action = action

            new_V[i, j] = max_value
            policy[i, j] = best_action
            delta = max(delta, abs(V[i, j] - new_V[i, j]))

        V[:] = new_V
        iteration += 1

        if delta < threshold:
            break

    return V, policy, iteration


What is the code doing?

Sets up all states in a 5x5 grid

Assigns rewards:

+10 to goal

-5 to bad states

-1 to all others

For each state:

Tries all 4 directions

Picks the best one

Repeats until values stop changing

In [8]:
V_star, policy_star, num_iters = value_iteration(V)

# Print value function
print("Optimal Value Function (V*):")
for row in V_star:
    print(['{0: >6.2f}'.format(v) for v in row])

# Print optimal policy
print("\nOptimal Policy (π*):")
for i in range(rows):
    row = ''
    for j in range(cols):
        if (i, j) == goal_state:
            row += '  G  '
        else:
            act = policy_star[i, j]
            symbol = {'up': '↑', 'down': '↓', 'left': '←', 'right': '→'}[act]
            row += f'  {symbol}  '
    print(row)


Optimal Value Function (V*):
[' -0.43', '  0.63', '  1.81', '  3.12', '  4.58']
['  0.63', '  1.81', '  3.12', '  4.58', '  6.20']
['  1.81', '  3.12', '  4.58', '  6.20', '  8.00']
['  3.12', '  4.58', '  6.20', '  8.00', ' 10.00']
['  4.58', '  6.20', '  8.00', ' 10.00', '  0.00']

Optimal Policy (π*):
  ↓    ↓    ↓    ↓    ↓  
  ↓    ↓    →    ↓    ↓  
  →    ↓    ↓    ↓    ↓  
  ↓    ↓    ↓    ↓    ↓  
  →    →    →    →    G  


Task 2: In-Place Value Iteration

Same idea, but we update values in-place instead of using a copy.

This makes the new values affect future updates immediately in the same loop.

🔁 In-Place Code Variation

In [10]:
def in_place_value_iteration(V, threshold=1e-4):
    policy = np.full((rows, cols), '', dtype=object)
    iteration = 0

    while True:
        delta = 0

        for state in states:
            if state == goal_state:
                continue

            i, j = state
            max_value = float('-inf')
            best_action = None

            for action, move in actions.items():
                ni, nj = i + move[0], j + move[1]
                if 0 <= ni < rows and 0 <= nj < cols:
                    next_state = (ni, nj)
                else:
                    next_state = (i, j)

                reward = get_reward(next_state)
                value = reward + gamma * V[next_state]

                if value > max_value:
                    max_value = value
                    best_action = action

            delta = max(delta, abs(V[i, j] - max_value))
            V[i, j] = max_value
            policy[i, j] = best_action

        iteration += 1

        if delta < threshold:
            break

    return V, policy, iteration
V_in_place = np.zeros((rows, cols))
V_in_place, policy_in_place, num_iters_in_place = in_place_value_iteration(V_in_place)
print("\nIn-Place Optimal Value Function (V*):")
for row in V_in_place:
    print(['{0: >6.2f}'.format(v) for v in row])
print("\nIn-Place Optimal Policy (π*):")
for i in range(rows):
    row = ''
    for j in range(cols):
        if (i, j) == goal_state:
            row += '  G  '
        else:
            act = policy_in_place[i, j]
            symbol = {'up': '↑', 'down': '↓', 'left': '←', 'right': '→'}[act]
            row += f'  {symbol}  '
    print(row)



In-Place Optimal Value Function (V*):
[' -0.43', '  0.63', '  1.81', '  3.12', '  4.58']
['  0.63', '  1.81', '  3.12', '  4.58', '  6.20']
['  1.81', '  3.12', '  4.58', '  6.20', '  8.00']
['  3.12', '  4.58', '  6.20', '  8.00', ' 10.00']
['  4.58', '  6.20', '  8.00', ' 10.00', '  0.00']

In-Place Optimal Policy (π*):
  ↓    ↓    ↓    ↓    ↓  
  ↓    ↓    →    ↓    ↓  
  →    ↓    ↓    ↓    ↓  
  ↓    ↓    ↓    ↓    ↓  
  →    →    →    →    G  


Comparison Between Classic and In-Place

| Feature         | Classic | In-Place          |
| --------------- | ------- | ----------------- |
| Uses copy of V? | Yes     | No                |
| Speed           | Slower  | Faster (usually)  |
| Memory          | More    | Less              |
| Accuracy        | Same    | Same (eventually) |
| Complexity      | Higher  | Lower             |


Problem 4 – Off-Policy Monte Carlo with Importance Sampling
🧠 What are we doing here?

We are using Monte Carlo method (learning from episodes), but we’re learning from a different policy than the one we want to learn!

That’s called Off-policy learning.

🎯 Goal:

Estimate the value of each state using:

A random behavior policy (used to explore)

A greedy target policy (what we want to learn)

Use importance sampling to correct the difference

📦 Environment: Same 5x5 Gridworld as Problem 3

Rewards:

+10 for goal (s4,4)

-5 for grey states (s0,4, s2,2, s3,0)

-1 for all others

Actions: up, down, left, right

γ (discount factor) = 0.9

🔍 Step-by-step Plan
1. Define Environment

Same as before

2. Behavior Policy (b)

Random — every action has equal chance

3. Target Policy (π)

Greedy — always picks best action based on current estimate

4. Generate Episodes

Use behavior policy to play full episodes (until goal)

5. Estimate Value Function

Use importance sampling to adjust the value based on how different b and π are.

# Off-policy MC with Importance Sampling

In [11]:
import numpy as np
import random

rows, cols = 5, 5
states = [(i, j) for i in range(rows) for j in range(cols)]

goal_state = (4, 4)
grey_states = [(0, 4), (2, 2), (3, 0)]

actions = ['up', 'down', 'left', 'right']
action_map = {
    'up': (-1, 0),
    'down': (1, 0),
    'left': (0, -1),
    'right': (0, 1)
}

gamma = 0.9

# Reward function
def get_reward(state):
    if state == goal_state:
        return 10
    elif state in grey_states:
        return -5
    else:
        return -1

# Behavior policy: Random
def behavior_policy():
    return random.choice(actions)

# Get next state
def next_state(state, action):
    i, j = state
    di, dj = action_map[action]
    ni, nj = i + di, j + dj
    if 0 <= ni < rows and 0 <= nj < cols:
        return (ni, nj)
    return state  # invalid move

# Generate one episode using behavior policy
def generate_episode():
    state = random.choice(states[:-1])  # start at any state except goal
    episode = []

    while state != goal_state:
        action = behavior_policy()
        next_s = next_state(state, action)
        reward = get_reward(next_s)
        episode.append((state, action, reward))
        state = next_s

    return episode

# Initialize
V = {s: 0 for s in states}
C = {s: 0 for s in states}
pi = {s: random.choice(actions) for s in states}

# Off-policy MC with Importance Sampling
def off_policy_mc(num_episodes=10000):
    for _ in range(num_episodes):
        episode = generate_episode()
        G = 0
        W = 1  # Importance Sampling Ratio

        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            G = gamma * G + reward
            C[state] += W
            V[state] += (W / C[state]) * (G - V[state])

            # Update greedy policy
            best_action = None
            best_value = float('-inf')
            for a in actions:
                s_prime = next_state(state, a)
                r = get_reward(s_prime)
                val = r + gamma * V[s_prime]
                if val > best_value:
                    best_value = val
                    best_action = a
            pi[state] = best_action

            if action != pi[state]:
                break  # stop updating if behavior and target policy differ
            W *= 1 / 0.25  # each action in behavior policy has prob = 0.25

    return V, pi


Run and Show Output

In [12]:
V_mc, pi_mc = off_policy_mc()

print("Estimated State Values (Monte Carlo):")
for i in range(rows):
    row = ''
    for j in range(cols):
        row += f'{V_mc[(i,j)]:6.2f} '
    print(row)

print("\nLearned Policy (π*) from Monte Carlo:")
for i in range(rows):
    row = ''
    for j in range(cols):
        if (i, j) == goal_state:
            row += '  G   '
        else:
            act = pi_mc[(i, j)]
            symbol = {'up': '↑', 'down': '↓', 'left': '←', 'right': '→'}[act]
            row += f'  {symbol}  '
    print(row)


Estimated State Values (Monte Carlo):
 -0.43  -0.99   1.81   1.11   1.07 
 -0.91   0.95   0.85   3.02   3.35 
 -0.15   0.88   3.10   3.71   5.87 
  1.13   2.93   3.62   6.29   7.58 
  2.19   4.30   5.89   7.60   0.00 

Learned Policy (π*) from Monte Carlo:
  ↓    →    ↑    ↓    ↓  
  →    ↓    →    ↓    ↓  
  →    ↓    →    ↓    ↓  
  →    ↓    →    ↓    ↓  
  →    →    →    →    G   


## Explanation of Code

We simulate 10,000 episodes using the random behavior policy

For each episode:

Calculate total reward (G)

Use importance sampling to adjust value estimates

Update greedy target policy based on best action

If behavior and target policy don’t match, stop updating

Final Comparison

| Feature                 | Value Iteration | Monte Carlo    |
| ----------------------- | --------------- | -------------- |
| Uses model of env?      | ✅ Yes           | ❌ No           |
| Needs episodes?         | ❌ No            | ✅ Yes          |
| Fast convergence?       | ✅ Yes           | ❌ Slower       |
| Needs transition probs? | ✅ Yes           | ❌ No           |
| Flexible for real data? | ❌ Less          | ✅ More         |
| Complexity              | Low             | Medium to High |


Final Thoughts

Value Iteration is powerful and fast when we know the environment.

Monte Carlo is flexible and works well even when we don’t know the environment's details.

Both methods help agents learn how to act optimally over time using rewards and experience.